# Tinychat AppView

Collect messages from jetstream, dispatch them. What does an app view do? See
[this](https://github.com/bluesky-social/atproto/discussions/2961)

In [ ]:
//| export

// based on https://docs.deno.com/examples/chat_app_tutorial/

export default class ChatServer {
  private connectedClients = new Map<string, WebSocket>();

  public handleConnection(ws: WebSocket) {
    const id = `${Math.random() * 100000}`;

    ws.onclose = () => {
      this.clientDisconnected(id);
    };

    this.connectedClients.set(id, ws);
    console.log(">>>>>>> connectedClients", this.connectedClients.size);
  }

  private clientDisconnected(id: string) {
    this.connectedClients.delete(id);
    console.log(`Client ${id} disconnected`);
  }

  public broadcast(message: string) {
    for (const client of this.connectedClients.values()) {
      console.log(">>>>>>> sending message to", client);
      client.send(message);
    }
  }
}

In [ ]:
//| export

// patch HTMX WS extension to allow content inspection and modification

// moved to static
const htmxWS = ``;

In [ ]:
//| export

import { Hono } from "hono";
import { HTTPException } from "hono/http-exception";
import { upgradeWebSocket } from "hono/deno";
// import { message } from "@tinychat/ui/message.tsx";
import { createMiddleware } from "hono/factory";
import { TinychatOAuthClient } from "tinychat/oauth.ts";
import { TinychatAgent } from "tinychat/agent.ts";
import { getDatabase } from "tinychat/db.ts"; 
import type { Database } from "tinychat/db.ts"; 

export type AppContext = {
  agent: () => Promise<TinychatAgent | undefined>;
  db?: Database | undefined;
};

export type HonoServer = Hono<{
  Variables: {
    ctx: AppContext;
  };
}>;

const app = new Hono();
const chatServer = new ChatServer();

app.use(
  "*",
  createMiddleware(async (c, next) => {
    const authorization = c.req.header("Authorization");
    const { client: oauthClient, user } = authorization
      ? await TinychatOAuthClient.fromAuthorizationHeader(authorization)
      : {};
    c.set("ctx", {
      oauthClient,
      session: undefined,
      agent: async () => await TinychatAgent.create(oauthClient, user),
      db: getDatabase(),
    });
    await next();
  }),
);

app.get("/", (c) => c.redirect("https://github.com/callmephilip/tinychat"));

app.get("/__test", (c) =>
  c.html(`<!DOCTYPE html>
<html>
<head>
    <title>HTMX Chat</title>
    <script src="https://unpkg.com/htmx.org@2.0.4"></script>
    <script>${htmxWS}</script>
    <style>
        .chat-container { max-width: 600px; margin: 20px auto; }
        .messages { height: 400px; overflow-y: auto; border: 1px solid #ccc; padding: 10px; margin-bottom: 10px; }
        .message { margin: 5px 0; padding: 5px; border-radius: 5px; background: #f0f0f0; }
        .input-form { display: flex; gap: 10px; }
        input { flex-grow: 1; padding: 5px; }
    </style>
</head>
<body>
    <div class="chat-container" hx-ext="ws" ws-connect="/ws">
        <div id="messages" class="messages">
        </div>
        <form class="input-form" ws-send>
            <input type="text" name="message" placeholder="Type a message..." autocomplete="off">
            <button type="submit">Send</button>
        </form>
    </div>
</body>
</html>`));

app.get(
  "/ws",
  upgradeWebSocket(() => {
    return {
      onOpen: (_, ws) => {
        if (!ws.raw) {
          return;
        }
        chatServer.handleConnection(ws.raw);
      },
    };
  }),
);

Hono {
  get: [Function (anonymous)],
  post: [Function (anonymous)],
  put: [Function (anonymous)],
  delete: [Function (anonymous)],
  options: [Function (anonymous)],
  patch: [Function (anonymous)],
  all: [Function (anonymous)],
  on: [Function (anonymous)],
  use: [Function (anonymous)],
  router: SmartRouter { name: "SmartRouter" },
  getPath: [Function: getPath],
  _basePath: "/",
  routes: [
    { path: "/*", method: "ALL", handler: [AsyncFunction (anonymous)] },
    { path: "/", method: "GET", handler: [Function (anonymous)] },
    { path: "/__test", method: "GET", handler: [Function (anonymous)] },
    {
      path: "/ws",
      method: "GET",
      handler: [AsyncFunction: UpgradeWebSocket]
    }
  ],
  errorHandler: [Function: errorHandler],
  onError: [Function: onError],
  notFound: [Function: notFound],
  fetch: [Function: fetch],
  request: [Function: request],
  fire: [Function: fire]
}

In [ ]:
//| export

import {
  NewChannelRecord,
  NewMembershipRecord,
  NewMessageRecord,
  NewServerRecord,
  startJetstream,
} from "tinychat/firehose.ts";
import { getProfile } from "tinychat/bsky.ts";

type AppViewShutdown = () => Promise<void>;
type AppViewContext = {
  database?: Database | undefined;
};

export const runAppView = (
  { database }: AppViewContext = {},
): AppViewShutdown => {
  const db = database || getDatabase();
  console.log("Starting appview with db", db);

  // Cleanup function
  const cleanup = () => {
    console.log("goodbye");
    Deno.removeSignalListener("SIGINT", cleanup);
    Deno.removeSignalListener("SIGTERM", cleanup);
    Deno.exit(0);
  };

  // Handle shutdown signals

  Deno.addSignalListener("SIGINT", cleanup);
  Deno.addSignalListener("SIGTERM", cleanup);

  console.log("Service started");

  const shutdownJetstream = startJetstream({
    onNewServer: async (m: NewServerRecord) => {
      const creator = m.did;
      const profile = await getProfile(creator);
      db.prepare(
        `
        INSERT INTO users (did, handle, display_name, avatar, description) VALUES (
          :did, :handle, :displayName, :avatar, :description
        ) ON CONFLICT(did) DO UPDATE SET
          handle = COALESCE(:handle, handle),
          display_name = COALESCE(:displayName, display_name),
          avatar = COALESCE(:avatar, avatar),
          description = COALESCE(:description, description
        )`,
      ).run({
        did: m.did,
        handle: profile.handle,
        displayName: profile.displayName,
        avatar: profile.avatar,
        description: profile.description,
      });
      db.prepare(`
      INSERT INTO servers (uri, name, creator) VALUES (
        :uri, :name, :creator
      )`).run({
        uri: m.uri,
        name: m.commit.record.name,
        creator: m.did,
      });
      db.prepare(
        `INSERT INTO server_memberships (user, server) VALUES (
          :creator, :server
        ) ON CONFLICT(user, server) DO NOTHING`,
      ).run({
        creator: m.did,
        server: m.uri,
      });
    },
    onNewChannel: (m: NewChannelRecord) => {
      db.prepare(
        `INSERT INTO channels (uri, name, server) VALUES (
          :uri, :name, :server
        ) ON CONFLICT(uri) DO NOTHING`,
      ).run({
        uri: m.uri,
        name: m.commit.record.name,
        server: m.commit.record.server,
      });
    },
    onNewMembership: (m: NewMembershipRecord) => {
      // add server memberships record
      try {
        db.prepare(
          `INSERT INTO server_memberships (user, server) VALUES (
          :creator, :server
        ) ON CONFLICT(user, server) DO NOTHING`,
        ).run({
          creator: m.did,
          server: m.commit.record.server,
        });
      } catch (e) {
        // normally this happens when creating a server and adding the creator to the server
        // membership gets processed before the server creation wraps up
        console.error("Error adding server membership", e);
      }
    },
    onNewMessage: (m: NewMessageRecord) => {
      db.prepare(
        `INSERT INTO messages (uri, channel, server, text, created_at) VALUES (
          :uri, :channel, :server, :text, :created_at
        )`,
      ).run({
        uri: m.uri,
        channel: m.commit.record.channel,
        server: m.commit.record.server,
        text: m.commit.record.text,
        created_at: m.commit.record.createdAt,
      });

      chatServer.broadcast(
        JSON.stringify({
          data: m.commit.record.text,
          html: `<div class="message">${m.commit.record.text}</div>`,
        }),
      );
    },
  });

  const server = Deno.serve(
    { port: parseInt(Deno.env.get("APPVIEW_PORT") || "8000") },
    app.fetch,
  );

  return async () => {
    try {
      shutdownJetstream();
      console.log("Shutting down server");
      await server.shutdown();
      console.log("Server shut down");
      Deno.removeSignalListener("SIGINT", cleanup);
      Deno.removeSignalListener("SIGTERM", cleanup);
    } catch (e) {
      console.error("Error shutting down server", e);
    }
  };
};

In [ ]:
const demo = () => {
  runAppView();

  return Deno.jupyter.html`
    <div style="padding: 20px; text-align: center;">
      <iframe
        width="800px"
        height="600px"
        src="http://localhost:8000"
      ></iframe>
    </div>
  `;
};

// Uncomment to run the demo
// demo();

In [ ]:
//| export

interface ServerData {
  uri: string;
  creator: string;
  name: string;
}

app.get("/xrpc/chat.tinychat.server.getServers", (c) => {
  const { db } = c.var.ctx;

  if (!db) {
    throw new HTTPException(500, { message: "DB not available" });
  }
  const servers = db.prepare(`SELECT * FROM servers`).all<ServerData>();
  const r = {
    servers: servers.map((s: ServerData) => ({
      uri: s.uri,
      creator: s.creator,
      name: s.name,
    })),
  };
  console.log("getServers", r);
  return c.json(r);
});

"";

""

## Test Appview

In [ ]:
import { TID } from "@atproto/common";
import { testClient } from "hono/testing";
import { assert, assertEquals } from "asserts";
import { sleep } from "tinychat/utils.ts";

Deno.test("/", async () => {
  // @ts-ignore cannot figure out type of test client
  const res = await testClient(app)["/"].$get();
  assertEquals(res.status, 302);
  // assertEquals(await res.json(), { status: "ok" });
});

// Deno.test("/xrpc/chat.tinychat.getServers", async () => {
//   // @ts-ignore cannot figure out type of test client
//   const res = await testClient(app)["/xrpc/chat.tinychat.getServers"].$get();
//   assertEquals(res.status, 200);
// });

Deno.test("test xrpc", async (t) => {
  const agent = await TinychatAgent.create();
  const serverName = `test-${TID.nextStr()}`;
  const db = getDatabase();
  const shutdown = runAppView({ database: db });

  // populate db, shall we?
  await agent.chat.tinychat.core.server.create(
    {
      repo: agent.agent.assertDid,
    },
    {
      name: serverName,
    }
  );


  await sleep(2000);

  await t.step("list available servers", async () => {
    const { data } = await agent.chat.tinychat.server.getServers({
      uris: [],
    });
    assert(data.servers.length > 0, "got a least 1 server");
    assert(data.servers.find(s => s.name === serverName), "found our server");
  });

  await shutdown();
  await sleep(2000);
});

Deno.test.ignore("test app view", async (t) => {
  const db = getDatabase();
  const shutdown = runAppView({ database: db });
  const serverName = `test-${TID.nextStr()}`;
  const agent = await TinychatAgent.create();
  const repo = agent.agent.assertDid;
  const receivedMessages: { data: string; html: string }[] = [];

  // create websocket connection to chat server
  const clientWS = new WebSocket("ws://localhost:8001/ws");
  clientWS.onmessage = (event) => {
    receivedMessages.push(JSON.parse(event.data));
  };

  // let's create a new chat server and watch it propagate through the system
  // should see new elements synced with the db

  let server = "no server yet";
  let channel = "no channel yet";

  await t.step("create server", async () => {
    const chatServer = await agent.chat.tinychat.core.server.create(
      {
        repo,
      },
      {
        name: serverName,
      }
    );
    server = chatServer.uri;

    await agent.chat.tinychat.core.membership.create(
      { repo },
      {
        server,
        createdAt: new Date().toISOString(),
      }
    );

    await sleep(2000);

    assert(
      db.prepare(`SELECT * FROM users`).all().length === 1,
      "user added to the db"
    );
    assert(
      db.prepare(`SELECT * FROM servers`).all().length === 1,
      "server added to the db"
    );
    assert(
      db.prepare(`SELECT * FROM server_memberships`).all().length === 1,
      "server membership added to the db"
    );
  });

  await t.step("create channel", async () => {
    const c = await agent.chat.tinychat.core.channel.create(
      { repo },
      {
        server,
        name: "general",
      }
    );
    channel = c.uri;

    await sleep(1000);

    assert(
      db.prepare(`SELECT * FROM channels`).all().length === 1,
      "channel added to the db"
    );
  });

  await t.step("send message", async () => {
    // add message
    await agent.chat.tinychat.core.message.create(
      { repo },
      {
        server,
        channel,
        text: "hello",
        createdAt: new Date().toISOString(),
      }
    );

    await sleep(1000);

    assert(
      db.prepare(`SELECT * FROM messages`).all().length === 1,
      "message added to the db"
    );
  });

  await t.step("create another server", async () => {
    await agent.chat.tinychat.core.server.create(
      { repo },
      { name: serverName + "2" }
    );

    await sleep(1000);

    assert(db.prepare(`SELECT * FROM servers`).all().length === 2);
  });

  await t.step("confirm messages get received over ws", () => {
    assert(receivedMessages.length === 1, "got one message");
    assert(
      typeof receivedMessages[0].data === "string",
      "message has a string data field"
    );
    assert(
      typeof receivedMessages[0].data === "string",
      "message has a string data field"
    );
  });

  // clean up and shutdown

  await shutdown();
  clientWS.close();
  await sleep(2000);
});